<img src="https://nds.iaea.org/relnsd/images/iaea.PNG" style="float:left; border-radius:6px">
<div style="margin-top:10px;font-size:22pt;font-weight:bold;">&nbsp;&nbsp; Livechart API + Data Science packages
    <div style="font-size:14pt; font-weight:normal; padding-left:100px;margin-bottom:-10px;margin-top:-10px">Version <span style="font-family:Monospace">0</span></div>
<!--span style="font-size:16pt; font-weight:normal"> version 0</span-->
</div>
<br>
<div style="border-bottom:1px solid black;max-width:700px;margin-top:-12px"></div>

<br>
This notebook makes use of<br>

**Pandas** : a library for data manipulation:
https://pandas.pydata.org
<br>
**Plotly** http://plotly.com : a package for interactive data visualization, 
<a href="http://plotly.com/python/getting-started/#jupyterlab-support-python-35">here</a> how to get it work with jupyter lab 
<br>

Write your feedback to nds.contact-point@iaea.org
 
## API Quick how-to
The service URL is  <b>nds.iaea.org/relnsd/v0/data?</b>
followed by parameters. For example:
<br> nds.iaea.org/relnsd/v0/data?<b>fields=decay_rads&nuclides=241am&rad_types=g</b>.<br>
The <a href=//nds.iaea.org/relnsd/vcharthtml/api_v0_guide.html><b>API v0 guide</b></a> gives the detailed description, here a short summary
<br><br>
<b>`fields=`</b> specifies what is retrieved  (at the bottom of this notebook there is the list of the columns for each choice)
<br> these are the possible options<br><br>
_ground_states <br> levels  <br> gammas  <br> decay_rads  <br> cumulative_fy  <br> independent_fy_ 
<br><br>
for _ground_states, levels, gammas, decay_rads_ it is mandatory to add the <b>nuclide</b> parameter<br>
`fields=levels&nuclides=135xe`
<br><br>
for _decay_rads_ also the parameter <b>rad_types</b> is mandatory. Values are a, bp, bm, g, e, x (alpha, beta+, beta-, gamma, electron, X-ray) 
<br>
`fields=decay_rads&nuclides=135xe&rad_types=bm`
<br>Be aware e aware of the fields <b>"p_energy" and
                        "decay"</b>. For
                    the same nuclide there can
                    be more than one decay mode, and, in case of metastable states, more states than the simple ground
                    state. This might result in gamma lines having the
                    same energy, but emitted by different decays 
<br><br>
for  _cumulative_fy, independent_fy_ one has to specify the <b>parent</b> and/or the <b>product</b>. Parents are _232th, 233u, 235u, 238u, 237np, 239pu, 241pu, 241am_ 
<br>
`fields=cumulative_fy&parents=233u&product=135xe `

<div style="margin-top:10px;font-size:18pt;font-weight:bold;">  Examples</div>
<br>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# the service URL
livechart = "https://nds.iaea.org/relnsd/v0/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))



## Plot &gamma; energy vs intensity for Am-241 decay

<b>fields=decay_rads</b> specifies to retrieve the decay radiations, <b>nuclides=241am</b>
specifies the parent of the decay (241am), <b>rad_types=g</b> for the radiation type (gamma). 
<br>
On the **plot** below, mouseover the points to see the data. Zoom an pan are enabled, and the option to save as png is is activated when the mouse is over the top-right

In [ ]:
# load data into a dataframe 
df = lc_pd_dataframe(livechart + "fields=decay_rads&nuclides=241am&rad_types=g")

df = df[pd.to_numeric(df['intensity'],errors='coerce').notna()] # remove blanks (unknown intensities)
df.intensity = df['intensity'].astype(float) # convert to numeric. Note how one can specify the field by attribute or by string 

fig = px.scatter(df, x="energy", y="intensity", log_y=True) # plot in log scale
fig.show()

## Plot Log FT vs Transition type for Nb-100 decay
The fields to be retieved are decay_rads, the nuclide is 100-Nb, and the radiation type is bm
<br>
In this case, only the relevant fields are loaded into the dataset; then data are grouped by transition, binned, and plotted

The transition types in the plot legend are not decoded. A stands for Allowed, 1NU for 1st non unique, ...

In [ ]:
url = livechart+"fields=decay_rads&nuclides=100nb&rad_types=bm"

df=lc_pd_dataframe(url)[['transition_type','log_ft']] # optional:load only transition_type and log_ft 

df = df[pd.to_numeric(df.log_ft,errors='coerce').notna()] # remove unknowns and convert to numeric values
df.log_ft = df.log_ft.astype(float)

ans = [pd.DataFrame(y) for x, y in df.groupby('transition_type', as_index=False)] # group log ft values by transition
fig = go.Figure()

for a in ans:
    ct = pd.cut(a.log_ft, bins=10)   # bin each grouping 
    tst = ct.apply(lambda x: x.mid).value_counts(sort=False)  # assign the mid value to each bin
    fig.add_trace(go.Scatter(x=tst.index.values, y = tst, mode="markers", line_shape='spline', name=a.transition_type.iloc[0]))
fig.show()

## Apply further filters once the data are loaded
The pandas <b>query</b> function acts on the loaded dataset.
<br>
To plot the Half-life of Ac isotopes, apply the *query* function to the column *symbol* to extract the *Ac* isotopes
```python
.query('symbol=="Ac"')
```


In [ ]:
url=livechart+"fields=ground_states&nuclides=all"

df=lc_pd_dataframe(url)[['symbol','n','half_life_sec']].query('symbol=="Ac"') # further filter

fig = px.scatter(df, x="n", y="half_life_sec", log_y=True)
fig.show()

## 3D and 2D plots of fission yields
Use the mouse to rotate and zoom

In [ ]:

url=livechart+"fields=cumulative_fy&parents=235u"
df = lc_pd_dataframe(url) [['z_daughter','a_daughter',"element_daughter",'cumulative_fast_fy']]
df = df[pd.to_numeric(df.cumulative_fast_fy,errors='coerce').notna()]
df.cumulative_fast_fy = df.cumulative_fast_fy.astype(float)
  

fig = go.Figure(data=[go.Scatter3d(
    x=df["z_daughter"],
    y=df["a_daughter"]-df["z_daughter"],
    z=df["cumulative_fast_fy"],
    mode='markers',
    text = df['a_daughter'].apply(str) + df["element_daughter"],
    marker=dict(
        size=7
        ,color=df["cumulative_fast_fy"] # array/list of values setting the color
        ,colorscale='Viridis'           # colorscale
    )
     

)])

camera = dict(                          # initial spacial settings 
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=0.1, y=2, z=-1.5)        # view it from the bottom
)

fig.update_layout(scene_camera=camera, margin=dict(l=0, r=0, b=0, t=0), height=900)
fig.show()

# 2D plot
fig2 = go.Figure()
fig2.add_trace(go.Scatter( x=df[df.columns[1]], y=df['cumulative_fast_fy'], mode="markers", text= df['a_daughter'].apply(str) + df["element_daughter"]))
fig2.update_layout( height=1000, xaxis_title="A", yaxis_title="Yield ") 
fig2.show()
        


## Column titles and samples of the available sets

**<span style="font-size:16pt">&bull;</span> Ground state**

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", 3)

# GS
df=lc_pd_dataframe(livechart+"fields=ground_states&nuclides=236np")
df

**<span style="font-size:16pt">&bull;</span> Level**
<br>
Note the "X" in the energy_shift field for the 2nd level, and the value in the "ripl_shift" column (see <a href="api_guide.html#fields">the API guide</a> for the meaning of the shift)

In [ ]:
# LEVEL
df=lc_pd_dataframe(livechart+"fields=levels&nuclides=236np")
df

**<span style="font-size:16pt">&bull;</span>Gamma**

In [ ]:
# GAMMA transitions
df=lc_pd_dataframe(livechart+"fields=gammas&nuclides=135xe")
df

**<span style="font-size:16pt">&bull;</span>Cumulative fission**

In [ ]:
# CUMULATIVE_FISSION
df=lc_pd_dataframe(livechart+"fields=cumulative_fy&products=135xe")
df

**<span style="font-size:16pt">&bull;</span>Independent fission**

In [ ]:
# INDEPENDENT FISSION
df=lc_pd_dataframe(livechart+"fields=independent_fy&parents=235u")
df

**<span style="font-size:16pt">&bull;</span>Decay radiation**

In [ ]:
# DECAY_RADIATION, type alpha
df=lc_pd_dataframe(livechart+"fields=decay_radiations&nuclides=238u&rad_types=a")
df

In [ ]:
# DECAY_RADIATION, type gamma
df=lc_pd_dataframe(livechart+"fields=decay_rads&nuclides=238u&rad_types=g")
df

In [ ]:
# DECAY_RADIATION, type x-ray
df=lc_pd_dataframe(livechart+"fields=decay_rads&nuclides=238u&rad_types=x")
df

In [ ]:
# DECAY_RADIATION, type beta-
df=lc_pd_dataframe(livechart+"fields=decay_rads&nuclides=135xe&rad_types=bm")
df

In [ ]:
# DECAY_RADIATION, type electron
df=lc_pd_dataframe(livechart+"fields=decay_rads&nuclides=238u&rad_types=e")
df

In [ ]:
# DECAY_RADIATION, type beta+/electron capture
df=lc_pd_dataframe(livechart+"fields=decay_rads&nuclides=236np&rad_types=bp")
df